### Create Dataframes

In [0]:
from pyspark.sql import functions as F, types as T

rows_customers = [
    (1,  "Asha",  "IN", True),
    (2,  "Bob",   "US", False),
    (3,  "Chen",  "CN", True),
    (4,  "Diana", "US", None),
    (None, "Ghost","UK", False),     # NULL key to demo null join behavior
]

rows_orders = [
    (101, 1,   120.0, "IN"),
    (102, 1,    80.0, "IN"),
    (103, 2,    50.0, "US"),
    (104, 5,    30.0, "DE"),         # no matching customer_id
    (105, 3,   200.0, "CN"),
    (106, None, 15.0, "UK"),         # NULL key won’t match
    (107, 3,    40.0, "CN"),
    (108, 2,    75.0, "US"),
]

schema_customers = T.StructType([
    T.StructField("customer_id", T.IntegerType(), True),
    T.StructField("name",        T.StringType(),  True),
    T.StructField("country",     T.StringType(),  True),
    T.StructField("vip",         T.BooleanType(), True),
])

schema_orders = T.StructType([
    T.StructField("order_id",    T.IntegerType(), True),
    T.StructField("customer_id", T.IntegerType(), True),
    T.StructField("amount",      T.DoubleType(),  True),
    T.StructField("country",     T.StringType(),  True),  # same column name to show collisions
])

df_customers = spark.createDataFrame(rows_customers, schema_customers)
df_orders    = spark.createDataFrame(rows_orders,    schema_orders)

display(df_customers)
display(df_orders)

customer_id,name,country,vip
1,Asha,IN,true
2,Bob,US,false
3,Chen,CN,true
4,Diana,US,null
null,Ghost,UK,false


order_id,customer_id,amount,country
101,1,120.0,IN
102,1,80.0,IN
103,2,50.0,US
104,5,30.0,DE
105,3,200.0,CN
106,null,15.0,UK
107,3,40.0,CN
108,2,75.0,US


### Inner Join

In [0]:
 o = df_orders.alias("o")
 o.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|     101|          1| 120.0|     IN|
|     102|          1|  80.0|     IN|
|     103|          2|  50.0|     US|
|     104|          5|  30.0|     DE|
|     105|          3| 200.0|     CN|
|     106|       NULL|  15.0|     UK|
|     107|          3|  40.0|     CN|
|     108|          2|  75.0|     US|
+--------+-----------+------+-------+



In [0]:
o, c = df_orders.alias("o"), df_customers.alias("c")

df_inner = o.join(c, on="customer_id", how="inner")
display(df_inner)

customer_id,order_id,amount,country,name,country,vip
1,101,120.0,IN,Asha,IN,true
1,102,80.0,IN,Asha,IN,true
2,103,50.0,US,Bob,US,false
3,105,200.0,CN,Chen,CN,true
3,107,40.0,CN,Chen,CN,true
2,108,75.0,US,Bob,US,false


### Disambiguate Columns

In [0]:
df_inner_clean = (
    o.join(c, on="customer_id", how="inner")
     .select(
        "order_id", "customer_id", "amount",
        F.col("o.country").alias("ship_country"),
        "name", F.col("c.country").alias("cust_country"), "vip"
     )
)
display(df_inner_clean)

order_id,customer_id,amount,ship_country,name,cust_country,vip
101,1,120.0,IN,Asha,IN,true
102,1,80.0,IN,Asha,IN,true
103,2,50.0,US,Bob,US,false
105,3,200.0,CN,Chen,CN,true
107,3,40.0,CN,Chen,CN,true
108,2,75.0,US,Bob,US,false


### Other Joins: Left, Full etc.

In [0]:
display(o.join(c, on="customer_id", how="left"))

customer_id,order_id,amount,country,name,country,vip
1,101,120.0,IN,Asha,IN,true
1,102,80.0,IN,Asha,IN,true
2,103,50.0,US,Bob,US,false
5,104,30.0,DE,null,null,null
3,105,200.0,CN,Chen,CN,true
null,106,15.0,UK,null,null,null
3,107,40.0,CN,Chen,CN,true
2,108,75.0,US,Bob,US,false


In [0]:
display(o.join(c, on="customer_id", how="full"))

customer_id,order_id,amount,country,name,country,vip
1,101,120.0,IN,Asha,IN,true
1,102,80.0,IN,Asha,IN,true
2,103,50.0,US,Bob,US,false
5,104,30.0,DE,null,null,null
3,105,200.0,CN,Chen,CN,true
null,106,15.0,UK,null,null,null
3,107,40.0,CN,Chen,CN,true
2,108,75.0,US,Bob,US,false
null,null,null,null,Ghost,UK,false
4,null,null,null,Diana,US,null


### Left Semi and Left Anti

In [0]:
display(o.join(c, on="customer_id", how="left_semi")) # orders with a known customer

customer_id,order_id,amount,country
1,101,120.0,IN
1,102,80.0,IN
2,103,50.0,US
3,105,200.0,CN
3,107,40.0,CN
2,108,75.0,US


In [0]:
display(o.join(c, on="customer_id", how="left_anti"))  # orphan orders (no matching customer)

customer_id,order_id,amount,country
5,104,30.0,DE
null,106,15.0,UK


### Multi Key Join

In [0]:
df_multi = o.join(c, on=["customer_id", "country"], how="inner")
display(df_multi)

customer_id,country,order_id,amount,name,vip
1,IN,101,120.0,Asha,true
1,IN,102,80.0,Asha,true
2,US,103,50.0,Bob,false
3,CN,105,200.0,Chen,true
3,CN,107,40.0,Chen,true
2,US,108,75.0,Bob,false
